In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Deep_learning_final

/content/drive/MyDrive/Colab Notebooks/Deep_learning_final


| **Transfer to Model** | **Accuracy Before Attack** | **Patch Size (3, 3)** | **Patch Size (5, 5)** | **Patch Size (7, 7)** | **Patch Size (16, 16)** |
|-----------------------|----------------------------|------------------------|------------------------|------------------------|-------------------------|
| **ResNet**           | 82.19%                    | 76.96%                | 78.19%                | 81.23%                | 87.67%                 |
| **DenseNet**         | 84.68%                    | 71.37%                | 72.42%                | 77.42%                | 87.26%                 |
| **VGG**              | 89.27%                    | 82.73%                | 74.85%                | 71.80%                | 85.52%                 |
| **MobileNet**        | 82.17%                    | 43.69%                | 51.06%                | 61.11%                | 82.62%                 |
| **EfficientNet**     | 79.71%                    | 60.12%                | 66.44%                | 74.11%                | 89.84%                 |



| **Transfer to Model** | **Patch Size (3, 3)** | **Patch Size (5, 5)** | **Patch Size (7, 7)** | **Patch Size (16, 16)** |
|-----------------------|------------------------|------------------------|------------------------|-------------------------|
| **ResNet**           | 76.96%                | 78.19%                | 81.23%                | 87.67%                 |
| **DenseNet**         | 71.37%                | 72.42%                | 77.42%                | 87.26%                 |
| **VGG**              | 82.73%                | 74.85%                | 71.80%                | 85.52%                 |
| **MobileNet**        | 43.69%                | 51.06%                | 61.11%                | 82.62%                 |
| **EfficientNet**     | 60.12%                | 66.44%                | 74.11%                | 89.84%                 |

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set up GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch
from torchvision import models
from torchvision.models import efficientnet_b0
import torch.nn as nn
# Define the model (same architecture as when you saved it)
num_classes = 10

#load resnet
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
resnet.load_state_dict(torch.load("cifar_resnet18_pretrained.pth"))
resnet = resnet.to(device)

#load densenet
densenet = models.densenet121(pretrained=False)  # Don't load the pre-trained weights initially
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet.load_state_dict(torch.load("cifar_densenet_pretrained.pth"))
densenet = densenet.to(device)

#load vgg
vgg = models.vgg16(pretrained=False)
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, num_classes)
vgg.load_state_dict(torch.load("cifar_vgg_pretrained.pth"))
vgg = vgg.to(device)

#load mobilenet
mobilenet = models.mobilenet_v2(pretrained=False)
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, num_classes)
mobilenet.load_state_dict(torch.load("cifar_mobilenet_pretrained.pth"))
mobilenet = mobilenet.to(device)

#load efficientnet
efficientnet = efficientnet_b0(pretrained=False)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, num_classes)
efficientnet.load_state_dict(torch.load("cifar_efficientnet_pretrained.pth"))
efficientnet = efficientnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-4-7363661e5dc5>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [12]:
def place_patch(img, patch):
    patch = patch.to(img.device)
    patched_img = img.clone()
    batch_size, _, h, w = patched_img.size()
    ph, pw = patch.size(1), patch.size(2)
    for i in range(batch_size):
        x_offset = torch.randint(0, h - ph + 1, (1,)).item()
        y_offset = torch.randint(0, w - pw + 1, (1,)).item()
        patched_img[i, :, x_offset : x_offset + ph, y_offset : y_offset + pw] = patch
    return patched_img

def evaluate_transferability(model, patch, dataloader):
    model.eval()
    untargeted_success = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            patched_images = place_patch(images.clone(), patch)
            outputs = model(patched_images)
            _, predicted = outputs.max(1)
            untargeted_success += (predicted != labels).sum().item()
            total += labels.size(0)
    return 100 * untargeted_success / total

In [14]:
# Load the patch
patch_path_list = ['Untargeted_adversarial_patch_3x3.pth', 'Untargeted_adversarial_patch_5x5.pth', 'Untargeted_adversarial_patch_7x7.pth', "Untargeted_adversarial_patch_16x16.pth"]
for patch_path in patch_path_list:
    print(f"Evaluating patch: {patch_path}")
    patch = torch.load(patch_path)

    # Evaluate transferability
    resnet_rate = evaluate_transferability(resnet, patch, testloader)
    print(f"Untargeted Attack Success Rate on ResNet: {resnet_rate:.2f}%")

    densenet_rate = evaluate_transferability(densenet, patch, testloader)
    print(f"Untargeted Attack Success Rate on DenseNet: {densenet_rate:.2f}%")

    vgg_rate = evaluate_transferability(vgg, patch, testloader)
    print(f"Untargeted Attack Success Rate on VGG: {vgg_rate:.2f}%")

    mobilenet_rate = evaluate_transferability(mobilenet, patch, testloader)
    print(f"Untargeted Attack Success Rate on MobileNet: {mobilenet_rate:.2f}%")

    efficientnet_rate = evaluate_transferability(efficientnet, patch, testloader)
    print(f"Untargeted Attack Success Rate on EfficientNet: {efficientnet_rate:.2f}%")

Evaluating patch: Untargeted_adversarial_patch_3x3.pth


<ipython-input-14-d933da13a658>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  patch = torch.load(patch_path)


Untargeted Attack Success Rate on ResNet: 76.96%
Untargeted Attack Success Rate on DenseNet: 71.37%
Untargeted Attack Success Rate on VGG: 82.73%
Untargeted Attack Success Rate on MobileNet: 43.69%
Untargeted Attack Success Rate on EfficientNet: 60.12%
Evaluating patch: Untargeted_adversarial_patch_5x5.pth
Untargeted Attack Success Rate on ResNet: 78.19%
Untargeted Attack Success Rate on DenseNet: 72.42%
Untargeted Attack Success Rate on VGG: 74.85%
Untargeted Attack Success Rate on MobileNet: 51.06%
Untargeted Attack Success Rate on EfficientNet: 66.44%
Evaluating patch: Untargeted_adversarial_patch_7x7.pth
Untargeted Attack Success Rate on ResNet: 81.23%
Untargeted Attack Success Rate on DenseNet: 77.42%
Untargeted Attack Success Rate on VGG: 71.80%
Untargeted Attack Success Rate on MobileNet: 61.11%
Untargeted Attack Success Rate on EfficientNet: 74.11%
Evaluating patch: Untargeted_adversarial_patch_16x16.pth
Untargeted Attack Success Rate on ResNet: 87.67%
Untargeted Attack Success

In [5]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

# Define CIFAR-10 class names
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class AdversarialAttackEvaluator:
    def __init__(self, model, device):
        self.model = model
        self.device = device

    def place_patch(self, img, patch, placement_strategy='random'):
        """
        Place patch on images with different placement strategies

        Args:
            img (torch.Tensor): Original images
            patch (torch.Tensor): Adversarial patch
            placement_strategy (str): Patch placement method ('random' or 'center')

        Returns:
            torch.Tensor: Images with applied patch
        """
        batch_size, _, h, w = img.size()
        ph, pw = patch.size(1), patch.size(2)

        if placement_strategy == 'random':
            # Random placement
            top = torch.randint(0, h - ph, (batch_size,)).to(self.device)
            left = torch.randint(0, w - pw, (batch_size,)).to(self.device)

            for i in range(batch_size):
                img[i, :, top[i]:top[i]+ph, left[i]:left[i]+pw] = patch
        elif placement_strategy == 'center':
            # Center placement
            top = (h - ph) // 2
            left = (w - pw) // 2
            img[:, :, top:top+ph, left:left+pw] = patch

        return img

    def evaluate_attack(self, dataloader, patch, target_class_idx):
        """
        Evaluate attack success for the given patch

        Args:
            dataloader (torch.utils.data.DataLoader): Test data loader
            patch (torch.Tensor): The adversarial patch
            target_class_idx (int): Target class index for attack (e.g., 'cat' class)

        Returns:
            float: Attack success rate
        """
        self.model.eval()
        correct = 0
        total = 0

        # Evaluate with different placement strategies
        placement_strategies = ['random', 'center']
        strategy_success_rates = []

        for strategy in placement_strategies:
            temp_correct = 0
            temp_total = 0

            with torch.no_grad():
                for images, labels in tqdm(dataloader):
                    images, labels = images.to(self.device), labels.to(self.device)
                    patched_images = self.place_patch(images.clone(), patch, strategy)

                    outputs = self.model(patched_images)
                    _, predicted = outputs.max(1)

                    # Calculate success for targeted attack
                    temp_correct += (predicted == target_class_idx).sum().item()
                    temp_total += labels.size(0)

            strategy_success_rate = 100 * temp_correct / temp_total
            strategy_success_rates.append(strategy_success_rate)
            print(f"Attack Success Rate ({strategy} placement): {strategy_success_rate:.2f}%")

        # Average success rate across strategies
        avg_success_rate = np.mean(strategy_success_rates)
        print(f"Average Attack Success Rate: {avg_success_rate:.2f}%")
        return avg_success_rate

In [ ]:
# Load adversarial patch
patch = torch.load('Untargeted_adversarial_patch_7x7.pth')  # Ensure this patch is of size 5x5
patch.requires_grad = False  # No gradient needed for evaluation

# Define target class index for 'cat'
target_class_idx = classes.index('cat')

#print
print("Attack on Resnet with target class cat")
# Create evaluator for ResNet
resnet_evaluator = AdversarialAttackEvaluator(resnet, device)

# Evaluate attack on ResNet
resnet_success_rate = resnet_evaluator.evaluate_attack(testloader, patch, target_class_idx)

#vgg
print("Attack on VGG with target class cat")
vgg_evaluator = AdversarialAttackEvaluator(vgg, device)
vgg_success_rate = vgg_evaluator.evaluate_attack(testloader, patch, target_class_idx)

#densenet
print("Attack on Densenet with target class cat")
densenet_evaluator = AdversarialAttackEvaluator(densenet, device)
densenet_success_rate = densenet_evaluator.evaluate_attack(testloader, patch, target_class_idx)

#mobilenet
print("Attack on Mobilenet with target class cat")
mobilenet_evaluator = AdversarialAttackEvaluator(mobilenet, device)
mobilenet_success_rate = mobilenet_evaluator.evaluate_attack(testloader, patch, target_class_idx)

#efficientnet
print("Attack on Efficientnet with target class cat")
efficientnet_evaluator = AdversarialAttackEvaluator(efficientnet, device)
efficientnet_success_rate = efficientnet_evaluator.evaluate_attack(testloader, patch, target_class_idx)